# **Classification of Shapes**

## **Importing modules**

In [ ]:
import os
import pathlib
import cv2
import glob
from PIL import Image

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix, classification_report
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import roc_auc_score, accuracy_score, roc_curve


import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1,l2
from tensorflow.keras.utils import to_categorical
#from tf_keras_vis.saliency import Saliency
#from tf_keras_vis.utils import normalize
#from tf_keras_vis.gradcam import Gradcam

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input


import random

from shape_load_image import create_data_sets


ModuleNotFoundError: ignored

In [ ]:
# Run this cell for more readable visuals 
large = 22; med = 16; small = 10
params = {'axes.titlesize': large,
          'legend.fontsize': med,
          'figure.figsize': (16, 10),
          'axes.labelsize': med,
          'axes.titlesize': med,
          'axes.linewidth': 2,
          'xtick.labelsize': med,
          'ytick.labelsize': med,
          'figure.titlesize': large}
plt.style.use('seaborn-whitegrid')
plt.rcParams.update(params)
%matplotlib inline

## **Loading the data and Preprocessing**

In [ ]:
# mounting the drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
ncopy=0

x,y,size_x,size_y=create_data_sets(ncopy)

In [ ]:
fig, ax = plt.subplots(1,5,figsize = (16,6))
for i in range(0,5):
  ax[i].imshow(x[i,:,:,0],cmap="gray")

## **Splitting into train, validation and test datasets**

In [ ]:
# Use the `ImageDataGenerator` function from keras to generate new images based on our existing ones
# Mention the preprocessing function as mobilenet's preprocess_input and specify a validation split of 20%
train_datagen=ImageDataGenerator(validation_split=0.2, preprocessing_function=preprocess_input, rescale=1./255.0) 

# Build your train_generator by specifying the directory the data_path variable defined above
# Mention target size as (224,224), color mode, class mode, batch_size, subset as 'training' and shuffle = True
train_generator=train_datagen.flow(x[:,:,:,0:3],y, batch_size=32, subset='training', shuffle=True)

# Build your validation_generator similar to the previous step 
# Specifying using the data_path variable defined above with subset as 'validation'
validation_generator=train_datagen.flow(x[:,:,:,0:3],y, batch_size=32, subset='validation', shuffle=True)


In [ ]:
for xt,yt in train_generator:
  print(xt.shape, yt.shape)
  break

fig, ax = plt.subplots(1,5,figsize = (16,6))
for i in range(0,5):
  ax[i].imshow(xt[i,:,:,2],cmap="gray")

fig, ax = plt.subplots(1,3,figsize = (16,6))
for i in range(0,3):
  ax[i].imshow(xt[0,:,:,i],cmap="gray")

## **Creating a model**

In [ ]:
def mobilenet_model():

  base_model=MobileNet(input_shape=(71,71,3), dropout=0.001, include_top=False,
    weights='imagenet', input_tensor=None, pooling=None,
    classifier_activation=None)
  
  x=base_model.output

  x=GlobalAveragePooling2D()(x)

# On top of mobile net, add a few dense layers with 'relu' activation

# Using functional API, add a dense layer with 1024 neurons 
  x=Dense(1024, activation="relu")(x)

# Add a dense layer with 512 neurons
  x=Dense(512, activation="relu")(x)

# Add a final layer with 2 neurons and softmax activation 
  preds=Dense(3, activation="softmax")(x)

  model=Model(base_model.inputs, preds)
  print(model.summary())

  for layer in model.layers[:10]:
    layer.trainable=False

  opt = Adam(learning_rate=0.0001)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])


  return model



In [ ]:
tf.keras.backend.clear_session()
model = mobilenet_model()

## **Evaluating the results with test data**

In [ ]:
# fit model
model_history = model.fit(train_generator, epochs=15, validation_data=validation_generator, verbose=1, steps_per_epoch=len(x_train) / 32,)

In [ ]:
# evaluate model
_, acc = model.evaluate(x_test, y_test, verbose=1)
print('> %.3f' % (acc * 100.0))

In [ ]:
# learning curves
# plot train and val acc as  a function of epochs
fig, ax = plt.subplots(1,2,figsize = (16,6))
ax[0].plot(model_history.history['loss'],color='#EFAEA4',label = 'Training Loss',linewidth=4.0)
ax[0].plot(model_history.history['val_loss'],color='#B2D7D0',label = 'Test Loss',linewidth=4.0)
ax[1].plot(model_history.history['accuracy'],color='#EFAEA4',label = 'Training Accuracy',linewidth=4.0)
ax[1].plot(model_history.history['val_accuracy'],color='#B2D7D0',label = 'Test Accuracy',linewidth=4.0)
ax[0].legend()
ax[1].legend()
ax[0].set_xlabel('Epochs', fontsize=18)
ax[1].set_xlabel('Epochs', fontsize=18)
ax[0].set_ylabel('Loss', fontsize=18)
ax[1].set_ylabel('Accuracy %', fontsize=18);
fig.suptitle('Model Training and Test Loss and Accuracy', fontsize = 24)

In [ ]:
from prettytable import PrettyTable
# your code here
print('CNN Regularized Model Summary')
model_table= PrettyTable()

model_table.field_names = ["Parameter", "Train", "Validation", "Difference (Train-Val)"]
model_table.add_row(["Loss", round(model_history.history['loss'][-10],2), round(model_history.history['val_loss'][-10],6), round(model_history.history['loss'][-10]-model_history.history['val_loss'][-10],6)])
model_table.add_row(["Accuracy", round(model_history.history['accuracy'][-10],6), round(model_history.history['val_accuracy'][-10],6), round(model_history.history['accuracy'][-10]-model_history.history['val_accuracy'][-10],6)])
print(model_table)

## **Error Analysis**

In [ ]:
print("\nPrediction Error on Test Set")
y_pred_test = model.predict(x_test)

In [ ]:

cnf_matrix = confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred_test, axis=1))
disp = ConfusionMatrixDisplay(confusion_matrix=cnf_matrix, display_labels=['Cylinder', 'Cuboid', 'Sphere'])
disp.plot()
plt.grid(None)
plt.title("Confusion Matrix", fontsize=24)
plt.show()
plt.savefig('ConfusionMatrix.pdf')

In [ ]:
print(classification_report(np.argmax(y_test, axis=1), np.argmax(y_pred_test, axis=1), target_names=['Cylinder', 'Cuboid', 'Sphere']))